# Modules

In [ ]:
# Required packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import pyDOE as doe
import sklearn.gaussian_process.kernels as skl
import pymc3 as pm
import theano.tensor as tt
import theano
import time
import seaborn as sns
from tqdm import tqdm
#import sys
from joblib import Parallel, delayed
import multiprocessing
#import psutil
import os
#import shutil
import copy
from matplotlib import gridspec
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

import pickle as pkl
import tensorflow as tf
import tensorflow_probability as tfp
#from scipy.optimize import minimize
#from scipy.optimize import fmin

# Function defs.

In [8]:
##### Run this block before any simulation or LDM code #####
def input_locations(d_input, n_input, lims, criterion = "c"): 
    """Simulation inputs generator
    
    The function generates inputs for model/observations according to latin hypercube design or uniform design
    within given range and dimension of inputs
    
    Args:
        d_input: The dimension of inputs to be generated
        n_input: The number of inputs
        lims: Ndarray with the range of inputs for each variable , each row is limits for each var
        criterion: "m", "c" for lating hypercube, "uniform" for uniform design
        
    Returns:
        inputs: n_input * d_input Ndarray with generated input points.
    """
    if criterion == "uniform":
        inputs = []
        for i in range(d_input):
            inputs = inputs + [np.linspace(lims[i, 0], lims[i, 1], int(n_input ** (1 / d_input)))]
            
        grid_list = np.meshgrid(*inputs)
        
        inputs = np.zeros((n_input, d_input))
        for i in range(d_input):
            inputs[:,i] = grid_list[i].flatten()        
        
    else:
        # Data generation
        inputs = doe.lhs(d_input, samples = n_input, criterion = criterion)

        # Transformation for the interval
        for i in range(d_input):
            inputs[:,i] = inputs[:,i] * (lims[i,1] - lims[i, 0]) + lims[i, 0]

    return inputs

def gp_mean_cov(input_obs, input_m, d_input, theta, kernels, means):
    """Creates mean vector and covariance matrix based on GP specifications for simulation
    
    The function generates the mean and convariance function of the prior GP that is 
    to be used for data generation of observations Y and model evaluations Z according to
    Kennedy & O'Hagan (2001) framework to calibration.
    
    Args:
        input_obs: Ndarray with observation inputs.
        input_m: Ndarray with model inputs.
        d_input: The dimension of model inputs.
        theta: True value of calibration parameter
        kernels: A dictionary containing 3 items: f, delta, and noise kernels
        means: A dictionary containing 2 items: f, delta
                
    Returns:
        M: Mean vector of MVN distribution
        K: Covariance matrix of MVN distribution
    """
    
    ##### Covariance part
    # Kernel input
    theta_stack = np.repeat(theta, len(input_obs))
    theta_stack = theta_stack.reshape((len(input_obs), input_m.shape[1] - d_input), order='F')
    kernel_input = np.concatenate((np.concatenate([input_obs, theta_stack], axis = 1), input_m), axis = 0)
    
    # Base kernel without noise for the model GP
    kernel_base = kernels["f"]
    K_base = kernel_base(kernel_input)
    
    # Model discrepancy kernel
    if kernels["delta"] != 0:
        
        zeros = np.zeros((input_m.shape[0], input_obs.shape[1]))
        delta_input = np.concatenate([input_obs, zeros])
        K_delta = kernels["delta"](delta_input)
        # Need to set the non-obs parts of the kernel to be zero
        K_delta[:, input_obs.shape[0]:] = 0
        K_delta[input_obs.shape[0]:, :] = 0
        K_base = K_base + K_delta
        
    # Add noise to the observations
    K = K_base + np.diag([kernels["sigma"] ** 2] * input_obs.shape[0] + [0] * input_m.shape[0])
    
    ##### Means part

    # Obs mean
    obs_mean = np.array([])
    input_obs_theta = np.concatenate([input_obs, theta_stack], axis = 1)
    for elem in input_obs_theta:
        # First input to mean is x, second input is theta
        obs_mean = np.append(obs_mean, means["f"](elem[:d_input], elem[d_input:]) + means["delta"](elem[:d_input]))
    
    # Model mean
    m_mean = np.array([])
    for elem in input_m:
        # First input to mean is x, second input is theta
        m_mean = np.append(m_mean, means["f"](elem[:d_input], elem[d_input:]))

    M = np.concatenate((obs_mean, m_mean))
    
    return M, K

def bijection_array(k, n, l): 
    """Calculates bijection of k in range 1,..., l * (2 * n - (l + 1)) / 2 
    mapped onto (i,j) where i in 1,..., n-j and j in 1,...,l
    
    Args:
        k: input
        n: number of observations
        l: truncation level of a vine
    
    Returns:
        i: index i
        j: index j
    """
        
    bijection_domain = int(round(l * (2 * n - (l + 1)) / 2))

    j = 0
    int_high_prev = 0
    int_high = 0
    while k > int_high:
        j = j + 1
        int_high_prev = int_high
        int_high = int_high + n - j

    i = k - int_high_prev
    
    return i, j

def partial_corr(i_low, i_hig, cond_list, data_input, kernels_input, corr_dict): 
    """Function calculates partial correlation rho_{i_low, i_hig: cond_list}
    based on the recursive formula. Here we assume i_low < i_hig and cond_list is
    ordered.
    
    Note:
    This is a utility function for the master functions defined below
    
    Args:
        i_low: First of the conditioned variables for partial correlation
        i_hig: Second of the conditioned variables for partial correlation
        cond_list: An ordered list of conditioning variables
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        corr_dict: a dictionary that is to be filled with calculated partial correlation
        
        NOTE: the entries in corr_dict have the following keys (strings) 
                i_low,i_high:cond_list[0],cond_list[-1] for conditioning list of size at leas 2
                i_low,i_high:cond_list[-1] for conditioning list of size 1
                i_low,i_high for conditioning list of size 0
    Returns:
        rho: Calculated partial correlation
        corr_dict: Updated correlation dictionary
    
    """
    
    if len(cond_list) > 0:
        
        corr_id = str(int(i_low)) + "," + str(int(i_hig)) + ":"
        if len(cond_list) > 1:
            corr_id = corr_id + str(int(cond_list[0])) + "," + str(int(cond_list[-1]))
        else:
            corr_id = corr_id + str(int(cond_list[0]))
            
        if corr_id not in corr_dict: 
            v = cond_list[-1]
            cond_list = cond_list[:-1]

            rho_i_j, corr_dict = partial_corr(i_low, i_hig, cond_list, data_input, kernels_input, corr_dict)

            rho_i_v, corr_dict = partial_corr(int(min(i_low,v)) , int(max(i_low,v)), cond_list,
                                                data_input, kernels_input, corr_dict)

            rho_v_j, corr_dict = partial_corr(int(min(i_hig,v)) , int(max(i_hig,v)), cond_list,
                                                data_input, kernels_input, corr_dict)

            rho = (rho_i_j - rho_i_v * rho_v_j) / np.sqrt((1 - rho_i_v ** 2) * (1 - rho_v_j ** 2))
            corr_dict[corr_id] = rho
        else:
            rho = corr_dict[corr_id]    
        
        
    else:
        corr_id = str(int(i_low)) + "," + str(int(i_hig))
        if corr_id not in corr_dict: 
                # All the neccessary inputs
            i_low_type = data_input["Response"][i_low - 1,:][1]
            i_hig_type = data_input["Response"][i_hig - 1,:][1]

            i_low_x = data_input["X"][i_low - 1,1:] 
            i_hig_x = data_input["X"][i_hig - 1,1:] 

            i_low_x_theta = np.concatenate((i_low_x, data_input["Theta"][i_low - 1,1:]))
            i_hig_x_theta = np.concatenate((i_hig_x, data_input["Theta"][i_hig - 1,1:]))

            # Here we need to distinguis between cases

            if (i_low_type == 1) & (i_hig_type == 0):
                # low is exp
                # high is model   

                #variances
                var_low = kernels_input["sigma"] ** 2
                if kernels_input["f"] != 0:
                    var_low = var_low + float(kernels_input["f"]([i_low_x_theta])) 
                    
                if kernels_input["delta"] != 0:
                    var_low = var_low + float(kernels_input["delta"]([i_low_x]))
                var_hig = float(kernels_input["f"]([i_hig_x_theta]))

                #correlation
                cov = float(kernels_input["f"](np.array([i_low_x_theta, i_hig_x_theta]))[0,1])
                rho = cov / (np.sqrt(var_hig * var_low))

            elif (i_low_type == 0) & (i_hig_type == 1):
                # low is model
                # hig is exp

                var_hig = float(kernels_input["f"]([i_hig_x_theta])) + kernels_input["sigma"] ** 2
                if kernels_input["delta"] != 0:
                    var_hig = var_hig + float(kernels_input["delta"]([i_hig_x]))
                var_low = float(kernels_input["f"]([i_low_x_theta]))

                #correlation
                cov = float(kernels_input["f"](np.array([i_low_x_theta, i_hig_x_theta]))[0,1])
                rho = cov / (np.sqrt(var_hig * var_low))

            elif (i_low_type == 0) & (i_hig_type == 0):
                # low is model
                # hig is model

                var_low = float(kernels_input["f"]([i_low_x_theta]))
                var_hig = float(kernels_input["f"]([i_hig_x_theta]))

                #correlation
                cov = float(kernels_input["f"](np.array([i_low_x_theta, i_hig_x_theta]))[0,1])
                rho = cov / (np.sqrt(var_hig * var_low))

            elif (i_low_type == 1) & (i_hig_type == 1):
                # low is exp
                # hig is exp

                #variances and covariances
                var_low = kernels_input["sigma"] ** 2
                var_hig = kernels_input["sigma"] ** 2
                cov = 0
                if kernels_input["f"] != 0:
                    var_low = float(kernels_input["f"]([i_low_x_theta])) + var_low
                    var_hig = float(kernels_input["f"]([i_hig_x_theta])) + var_hig
                    cov = float(kernels_input["f"](np.array([i_low_x_theta, i_hig_x_theta]))[0,1])
                if kernels_input["delta"] != 0:
                    var_low = var_low + float(kernels_input["delta"]([i_low_x]))
                    var_hig = var_hig + float(kernels_input["delta"]([i_hig_x]))
                    cov = float(cov + kernels_input["delta"](np.array([i_low_x, i_hig_x]))[0,1])

                rho = cov / (np.sqrt(var_hig * var_low))

            corr_dict[corr_id] = rho
        else:
            rho = corr_dict[corr_id]
    return rho, corr_dict


def partial_corr_c_master(i, j, data_input, kernels_input):
    """Function calculates partial correlation for the C-vine case.
    
    NOTE: This function is slightly redundant, I am not sure if I will actually need to use it
    
    Args:
        i: Index i from the bijection
        j: Index j from the bijection
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        
    Returns:
        rho: Calculated partial correlation for a Gaussian C-Vine to be calucalted
        corr_dict: A dictionary with all the partial correlations obrained through the recursion.
        
    NOTE: See partial_corr function for beter description of corr_dict
    """
    
    cond_list = [i + 1 for i in range(j - 1)]
    corr_dict = {}
    i_low = j
    i_hig = j + i
    
    rho, corr_dict = partial_corr(i_low, i_hig, cond_list, data_input, kernels_input, corr_dict)
    return rho, corr_dict

def partial_corr_d_master(i, j, data_input, kernels_input): 
    """Function calculates partial correlation for the D-vine case.
    
    NOTE: This function is slightly redundant, I am not sure if I will actually need to use it
    
    Args:
        i: Index i from the bijection
        j: Index j from the bijection
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        
    Returns:
        rho: Calculated partial correlation for a Gaussian D-Vine to be calucalted
        corr_dict: A dictionary with all the partial correlations obrained through the recursion.
        
    NOTE: See partial_corr function for beter description of corr_dict
    
    """
    
    cond_list = [i for i in range(i + 1 ,i + j)]
    corr_dict = {}
    i_low = i
    i_hig = j + i

    rho, corr_dict = partial_corr(i_low, i_hig, cond_list, data_input, kernels_input, corr_dict)
    return rho, corr_dict

def bivariate_gauss_copula(i, j, data_input, kernels_input, means_input, vine_type = "C"): 
    """Evaluates bivariate gaussian copula density for given values of i,j and vine copula type.
    
    Args:
        i: Output from bijection that uniquely identifies first datapoint
        j: Output from bijection that uniquely identifies second datapoint
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        means_input: A dictionary containing 2 items: f, delta
        vine_type: "C" or "D" depending on which wine copule we want to consider
        
    Returns:
        copula_density_eval: Evaluated bivariate gaussian copula    
    """
    
    # Calculating the corr_dict
    if vine_type == "C":
        cond_list = [i + 1 for i in range(j - 1)]
        i_low = j
        i_hig = j + i
        rho, corr_dict = partial_corr_c_master(i, j, data_input, kernels_input)
    elif vine_type == "D":
        cond_list = [i for i in range(i + 1 ,i + j)]
        i_low = i
        i_hig = j + i
        rho, corr_dict = partial_corr_d_master(i, j, data_input, kernels_input)
        
    # Based on a truncation level, what is the size of conditioning set?
    if len(cond_list) > 0:

        v = cond_list[-1]
        cond_list_h = cond_list[:-1]

        F_low = h_function(i_low, v, cond_list_h, data_input, kernels_input, means_input, corr_dict)
        F_hig = h_function(i_hig, v, cond_list_h, data_input, kernels_input, means_input, corr_dict)

        corr_low = min(i_low, i_hig)
        corr_hig = max(i_low, i_hig)
        
        # conditional indicator
        corr_id = str(int(corr_low)) + "," + str(int(corr_hig)) + ":" + str(int(cond_list[0]))
        if len(cond_list) > 1:
             corr_id = corr_id + "," + str(int(v))

        rho = corr_dict[corr_id]

    else:
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type = std_transform(i_low, i_hig, data_input, kernels_input, means_input)

        F_low = sp.stats.norm.cdf(Y_low_std)
        F_hig = sp.stats.norm.cdf(Y_hig_std)
 

        corr_low = min(i_low, i_hig)
        corr_hig = max(i_low, i_hig)

        # conditional indicator
        corr_id = str(int(corr_low)) + "," + str(int(corr_hig))
        rho = corr_dict[corr_id]

    copula_density_eval = b_gaussian_c_pdf(F_low, F_hig, rho)
    
    return copula_density_eval

def h_function(i_low, i_hig, cond_list, data_input, kernels_input, means_input, corr_dict): #Works for general case
    """Calculates h(u_{i_low}, u_{i_hig}) parameterized by rho_{i_low,i_hig:cond_list} recuresively
    
    NOTE: i_low, i_hig have slightly different meaning that in the case of partial correlation
    because here oreder matters so i_low, i_hig here correspinds to ored of arguments
    
    Args:
        i_low: First argument index for h
        i_hig: Second argument index for h
        cond_list: A list of conditioning variables for h viz rho in description
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        means_input: A dictionary containing 2 items: f, delta
        corr_dict: A dictionary containg all the partial correlations precalculated using parial_corr fcn
        
    Returns:
        h: Calculated value of h function
    """
    
    if len(cond_list) > 0:

        v = cond_list[-1]
        cond_list_h = cond_list[:-1]
        
        F_low = h_function(i_low, v, cond_list_h, data_input, kernels_input, means_input, corr_dict)
        F_hig = h_function(i_hig, v, cond_list_h, data_input, kernels_input, means_input, corr_dict)
        
        corr_low = min(i_low, i_hig)
        corr_hig = max(i_low, i_hig)
        
        # conditional indicator 
        corr_id = str(int(corr_low)) + "," + str(int(corr_hig)) + ":" + str(int(cond_list[0]))
        if len(cond_list) > 1:
            corr_id = corr_id + "," + str(int(v))

        rho = corr_dict[corr_id]
        h = sp.stats.norm.cdf((sp.stats.norm.ppf(F_low) - rho * sp.stats.norm.ppf(F_hig)) / np.sqrt(1 - rho ** 2))
             
    else:
            
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type = std_transform(i_low, i_hig, data_input, kernels_input, means_input)

        corr_low = min(i_low, i_hig)
        corr_hig = max(i_low, i_hig)

        
        rho = corr_dict[str(int(corr_low)) + "," + str(int(corr_hig))]
        h = sp.stats.norm.cdf((Y_low_std - rho * Y_hig_std) / np.sqrt(1 - rho ** 2))
        
    return h

def vine_p(i, j, l, n, data_input, kernels_input, means_input, vine_type = "C"):
    """Function calculates the p (as defined in Kejzlar and Maiti (2020))
    function based on the vine copula with given truncation
    
    Note:
    - this is the "p" for the Algorithm 1 and for Algorithm 1 + Control Variates 
    - p under Rao-Blackwellization defined below
    - this is version without Rao-Blackwellization
    
    Args:
        i: Index i from the bijection
        j: Index j from the bijection
        l: Truncation level
        n: Sample size
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        means_input: A dictionary containing 2 items: f, delta
        vine_type: "C" or "D" depending on which wine copule we want to consider
        
    Returns:
        p_eval: evaluated p_function
    """
    
    
    if vine_type == "D":
        if j > l: # This conditional is unnecesary since it is taken care of by the bijeciton function
            c_i_j = 1.0
        else:
            c_i_j = bivariate_gauss_copula(i, j, data_input, kernels_input, means_input, vine_type = "D")

        # definition of a_i multiplicatior
        a = 2 * l
        if i <= l:
            a = a - (l + 1 - i)
        if i > (n - l):
            a = a - (l - n + i)

        # definition of b_{i+j} multiplicator
        b = 2 * l
        if (i + j) <= l:
            b = b - (l + 1 - j - i)
        if (i + j) > (n - l):
            b = b - (l - n + j + i)

        # Standardized data
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type  = std_transform(i, j + i, data_input, kernels_input, means_input)
    else:
        if j > l: # See above
            c_i_j = 1.0
        else:
            c_i_j = bivariate_gauss_copula(i, j, data_input, kernels_input, means_input, vine_type = "C")

        # definition of a_i multiplier 
        a = n - 1

        # definition of b_{j+i}
        b = l
        if (j + i) <= l:
            b = b + (n - 1 - l)
        
        
        # Standardized data
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type = std_transform(j, j + i, data_input, kernels_input, means_input)
        
    a = float(a)
    b = float(b)
    
    p_eval = np.log(c_i_j) + (1 / a) * np.log(sp.stats.norm.pdf(Y_low_std)) + (1 / b) * np.log(sp.stats.norm.pdf(Y_hig_std))

    return p_eval

def data_type(i_low, i_hig, data_input):
    """Function extracts the type of data indexed by i_low and i_hig
    
     Args:
        i_low: First argument index
        i_hig: Second argument index
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
               
    Returns:
        i_low_type: 1 for y, 0 for z
        i_hig_type: 1 for y, 0 for z
    """
    
    i_low_type = data_input["Response"][i_low - 1,:][1]
    i_hig_type = data_input["Response"][i_hig - 1,:][1]
    
    return i_low_type, i_hig_type

def std_transform(i_low, i_hig, data_input, kernels_input, means_input): 
    """Standardize the data points according to the GP specification.
    
    Args:
        i_low: First argument index
        i_hig: Second argument index
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        means_input: A dictionary containing 2 items: f, delta
            
    Returns:
        Y_low_std: Standardized value with index i_low
        Y_hig_std: Standardized value with index i_hig 
        var_low: Variance of data point with index i_low
        var_hig: Variance of data point with index i_hig
        i_low_type: 1 for y, 0 for z
        i_hig_type: 1 for y, 0 for z
    """
    
    i_low_type = data_input["Response"][i_low - 1,:][1]
    i_hig_type = data_input["Response"][i_hig - 1,:][1]

    i_low_x = data_input["X"][i_low - 1,1:] 
    i_hig_x = data_input["X"][i_hig - 1,1:]

    i_low_y = data_input["Response"][i_low - 1,2]
    i_hig_y = data_input["Response"][i_hig - 1,2]

    i_low_theta = data_input["Theta"][i_low - 1,1:]
    i_hig_theta = data_input["Theta"][i_hig - 1,1:]

    # The basic simulation considers kernels without theta in them!
    i_low_x_theta = np.concatenate((i_low_x, data_input["Theta"][i_low - 1,1:]))
    i_hig_x_theta = np.concatenate((i_hig_x, data_input["Theta"][i_hig - 1,1:]))

        # Uniform inputs calculation

    if i_low_type == 1:
            # low is exp      
            # Varince
        var_low = kernels_input["sigma"] ** 2
        if kernels_input["f"] != 0:
            var_low = var_low + float(kernels_input["f"]([i_low_x_theta]))
        if kernels_input["delta"] != 0:
              var_low = float(var_low + float(kernels_input["delta"]([i_low_x])))
            # Mean
        mean_low = float(means_input["f"](i_low_x, i_low_theta) + means_input["delta"](i_low_x))

    elif i_low_type == 0:
            # low is model      
            # Variance
        var_low = float(kernels_input["f"]([i_low_x_theta]))
            # Mean
        mean_low = float(means_input["f"](i_low_x, i_low_theta))

    if i_hig_type == 1:
            # hig is exp     
            # Varince
        var_hig = kernels_input["sigma"] ** 2
        if kernels_input["f"] != 0:
            var_hig = float(kernels_input["f"]([i_hig_x_theta])) + var_hig
        if kernels_input["delta"] != 0:
            var_hig = float(var_hig + float(kernels_input["delta"]([i_hig_x])))
            # Mean
        mean_hig = float(means_input["f"](i_hig_x, i_hig_theta) + means_input["delta"](i_hig_x))

    elif i_hig_type == 0:
            # low is model    
            # Variance
        var_hig = float(kernels_input["f"]([i_hig_x_theta]))
            # Mean
        mean_hig = float(means_input["f"](i_hig_x, i_hig_theta))
        
        # USE logarith for stability
    Y_low_std = np.sign(i_low_y - mean_low) * np.exp(np.log(np.abs(i_low_y - mean_low)) - np.log(var_low) / 2)
    Y_hig_std = np.sign(i_hig_y - mean_hig) * np.exp(np.log(np.abs(i_hig_y - mean_hig)) - np.log(var_hig) / 2)
        
    return Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type

def b_gaussian_c_pdf(u1, u2, rho):
    """Calculates the actual bivariate gaussian copula density.
    
    Args:
        u1: The value of cdf for first arg
        u2: the value of cdf for second arg
        
    Returns:
        density: Evaluated bivariate gaussian copula density    
    """
    w1 = sp.stats.norm.ppf(u1)
    w2 = sp.stats.norm.ppf(u2)
    
    density = (1 / np.sqrt(1 - rho ** 2)) * np.exp(- ((rho ** 2) * (w1 ** 2 + w2 ** 2) - 2 * rho * w1 * w2) / (2 * (1 - rho ** 2)))
    return density

##### Distribution family classes that are used as 1) Variational fmailies, 2) Overdispersed familiees, 3) Priors
# Each class contains the following methods:
#
# __init__(self, param, init)
#
# sample(self, n_size)
#
# log_pdf(self, theta)
#
# log_grad_pdf(self, theta)

class gaussian_mean_field_family:
    """Class for a mean field gaussian family of arbitrary dimension"""
    
    def __init__(self, param, dim):
        """The parameters for the Gaussian mean field family are passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["mu"] is a 1-dim Ndarray with the means of independent Gaussian distributions
                 - param["sigma"] is a 1-dim Ndarray with the np.log(SDs) of independent Gaussian distributions
            dim: dimension of the family
        """
        
        self.mu = param["mu"]
        self.sigma = param["sigma"]
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gaussian mean_field variational family
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            sample[:, i] = np.random.normal(loc = self.mu[i], scale = self.sigma[i], size = n_size)     
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gaussian family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            density = sp.stats.norm(loc = self.mu[i], scale = self.sigma[i])
            log_dens = log_dens + np.log(density.pdf(theta[i]))       
        return log_dens
    
    def grad_log_pdf(self, theta):
        """Calculates the value of score function at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(2 * dim) where gradient[:dim] are the values of score function
                    w.r.t mu and gradient[dim:] are the values of score function w.r.t sigma
        """
        
        gradient = np.zeros(self.dim * 2)
        for i in range(self.dim):
            # score function w.r.t. mean
            gradient[i] = (theta[i] - self.mu[i]) / (self.sigma[i] ** 2)
            # score function w.r.t. SD
            gradient[i + self.dim] = - 1 / self.sigma[i] + ((theta[i] - self.mu[i]) ** 2) / (self.sigma[i] ** 3)
        return gradient
    
class gamma_mean_field_family:
    """Class for a mean field gamma family of arbitrary dimension with np.log(param) parametrization
      
      Gamma parametrization:
          
          f(x|alpha, beta) = \frac{beta ^ alpha}{Gamma(alpha)} * x ^{alpha - 1} * \exp{- beta * x}
          
          E(X) = alpha / beta
          Var(X) = alpha / (beta^2)
          
    """
    
    def __init__(self, param, dim):
        """The parameters for the gamma mean field family are passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["alpha"] is a 1-dim Ndarray with the np.log(alpha) parameters of independent Gamma distributions
                 - param["beta"] is a 1-dim Ndarray with the np.log(beta) parameters of independent Gamma distributions
            dim: Dimension of the family
        """
        
        self.alpha = param["alpha"]
        self.beta = param["beta"]
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gamma mean field variational family
        
        Note: np.random.gamma has parametrization with shape = alpha, scale = 1 / beta
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            sample[:, i] = np.random.gamma(shape = self.alpha[i], scale = self.beta[i], size = n_size)
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gamma family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            log_dens = log_dens + (self.alpha[i] - 1) * np.log(theta[i]) - theta[i] / self.beta[i] - self.alpha[i] * np.log(self.beta[i]) - np.log(sp.special.gamma(self.alpha[i]))
    
        return log_dens
    
    def grad_log_pdf(self, theta):
        """Calculates the value of score function at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(2 * dim) where gradient[:dim] are the values of score function
                    w.r.t alpha and gradient[dim:] are the values of score function w.r.t beta
        """
        
        gradient = np.zeros(self.dim * 2)
        for i in range(self.dim):    
            # score function w.r.t. alpha
            gradient[i] = np.log(theta[i]) - np.log(self.beta[i]) - sp.special.digamma(self.alpha[i])
            # score function w.r.t. beta
            gradient[i + self.dim] = theta[i] / (self.beta[i] ** 2) - self.alpha[i] / self.beta[i]
        return gradient
    
class gaussian_mean_field_family_lambda_param:
    """Class for a mean field gaussian family of arbitrary dimension with scale parametrization 
    
    lambda = log(exp(sigma) - a)
    
    """
    def __init__(self, param, dim):
        """The parameters for the Gaussian mean field family aare passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["mu"] is a 1-dim Ndarray with the means of independent Gaussian distributions
                 - param["sigma"] is a 1-dim Ndarray with the transformed sd of independent Gaussian distributions
                 - param["a"] see parametrization
            dim: Dimension of the family
        """
        
        self.mu = param["mu"]
        self.sigma = param["sigma"]
        self.a = param["a"]
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gaussian mean_field variational family
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            sd = np.log(np.exp(self.sigma[i]) + self.a)
            sample[:, i] = np.random.normal(loc = self.mu[i], scale = sd, size = n_size)     
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gaussian family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            sd = np.log(np.exp(self.sigma[i]) + self.a)
            density = sp.stats.norm(loc = self.mu[i], scale = sd)
            log_dens = log_dens + np.log(density.pdf(theta[i]))       
        return log_dens
    
    def grad_log_pdf(self, theta):
        """Calculates the value of score function at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(2 * dim) where gradient[:dim] are the values of score function
                    w.r.t mu and gradient[dim:] are the values of score function w.r.t sigma
        """
        
        gradient = np.zeros(self.dim * 2)
        for i in range(self.dim):
            # score function w.r.t. mean
            gradient[i] = - (self.mu[i] - theta[i]) / (np.log(np.exp(self.sigma[i]) + self.a) ** 2)
            # score function w.r.t. lambda
            gradient[i + self.dim] = np.exp(self.sigma[i]) * (- 1 / (np.log(self.a + np.exp(self.sigma[i])) * (self.a + np.exp(self.sigma[i]))) + \
                                                             ((self.mu[i] - theta[i]) ** 2) / ((self.a + np.exp(self.sigma[i])) * np.log(self.a + np.exp(self.sigma[i])) ** 3))
        return gradient

    
class gamma_mean_field_family_lambda_param:
    """Class for a mean field gamma family of arbitrary, parametrized with mean and standard deviation
    
    
    lambda_alfa = log(exp(alfa) - a)
    lambda_beta = log(exp(alfa) - a)
    
    where alpha corresponds to the mean of the gamma distribution and beta corresponds to the std of the gamma dist
      
      Gamma parametrization:
          
          f(x|alpha, beta) = \frac{beta ^ alpha}{Gamma(alpha)} * x ^{alpha - 1} * \exp{- beta * x}
          
          E(X) = alpha / beta
          Var(X) = alpha / (beta^2)
          
    """
    
    def __init__(self, param, dim):
        """The parameters for the gamma mean field family are passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["alpha"] is a 1-dim Ndarray with the mean paramteres of the independent Gamma distribution
                 - param["beta"] is a 1-dim Ndarray with the stds of independent Gamma distributions
                 - param["a"] see parametrization
            dim: Dimension of the family
        """
        
        self.alpha = param["alpha"]
        self.beta = param["beta"]
        self.a = param["a"]
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gamma mean field variational family
        
        Note: np.random.gamma has parametrization with shape = alpha, scale = 1 / beta
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            mu = np.log(np.exp(self.alpha[i]) + self.a)
            sigma = np.log(np.exp(self.beta[i]) + self.a)
            alpha = float(mu/sigma) ** 2
            beta = float(mu/ (sigma**2))
            tfd = tfp.distributions
            tfdGamma = tfd.Gamma(concentration=alpha, rate=beta)
            samples = tfdGamma.sample(n_size)
            sample[:, i] = samples.numpy()
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gamma family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            
            mu = np.log(np.exp(self.alpha[i]) + self.a)
            sigma = np.log(np.exp(self.beta[i]) + self.a)
            alpha = float(mu/sigma) ** 2
            beta = float(mu/ (sigma**2))
            tfd = tfp.distributions
            tfdGamma = tfd.Gamma(concentration=alpha, rate=beta)
            log_dens = log_dens + float(tfdGamma.log_prob(theta[i]))
        return log_dens
    
    def grad_log_pdf(self, theta):
        """Calculates the value of score function at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(2 * dim) where gradient[:dim] are the values of score function
                    w.r.t alpha and gradient[dim:] are the values of score function w.r.t beta
        """
        
        gradient = np.zeros(self.dim * 2)
        for i in range(self.dim):    
            l_m = self.alpha[i]
            l_s = self.beta[i]
            # score function w.r.t. lambda_alpha
            g1m = (np.log(self.a + np.exp(l_s)) ** 2) * (self.a + np.exp(l_m))
            g2m = np.log(self.a + np.exp(l_m))
            gradient[i] = (np.exp(l_m) * g2m - theta[i] * np.exp(l_m) - 2 * np.exp(l_m) * sp.special.digamma((g2m / np.log(self.a + np.exp(l_s))) ** 2) + \
                          2 * np.exp(l_m) * np.log(theta[i]) * g2m + 2 * np.log(g2m / (np.log(self.a + np.exp(l_s)) ** 2)) * np.exp(l_m) * g2m) / g1m
            
            # score function w.r.t. lambda_beta
            g1l = (np.log(self.a + np.exp(l_s)) ** 3) * (self.a + np.exp(l_s))
            g2l = np.log(self.a + np.exp(l_m)) ** 2
            gradient[i + self.dim] = (2 * np.exp(l_s) * sp.special.digamma(g2l / (np.log(self.a + np.exp(l_s)) ** 2)) * g2l - \
                                     2 * np.log(np.log(self.a + np.exp(l_m)) / (np.log(self.a + np.exp(l_s)) ** 2)) * np.exp(l_s) * g2l -\
                                     2 * np.exp(l_s) * g2l - 2 * np.exp(l_s) * np.log(theta[i]) * g2l + 2 * theta[i] * np.exp(l_s) * np.log(self.a + np.exp(l_m))) / g1l
        return gradient
    
    
class gaussian_mean_field_family_lambda_param_overdisp:
    """Class for a mean field overdispersed gaussian family of arbitrary dimension with scale parametrization 
    
    lambda = log(exp(sigma) - a)
    
    """
    def __init__(self, param, dim, tau):
        """The parameters for the Gaussian mean field family aare passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["mu"] is a 1-dim Ndarray with the means of independent Gaussian distributions
                 - param["sigma"] is a 1-dim Ndarray with the np.log(SDs) of independent Gaussian distributions
                 - param["a"] see parametrization 
            dim: Dimension of the family
            tau: Dispersion coefficient >= 1
        """
        
        self.mu = param["mu"]
        self.sigma = param["sigma"]
        self.a = param["a"]
        self.tau = tau
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gaussian mean_field variational family
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            sd = np.log(np.exp(self.sigma[i]) + self.a) * np.sqrt(self.tau[i])
            sample[:, i] = np.random.normal(loc = self.mu[i], scale = sd , size = n_size)     
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gaussian family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            sd = np.log(np.exp(self.sigma[i]) + self.a) * np.sqrt(self.tau[i])
            density = sp.stats.norm(loc = self.mu[i], scale = sd)
            log_dens = log_dens + np.log(density.pdf(theta[i]))       
        return log_dens
    
    def grad_log_pdf_tau(self, theta):
        """"Calculate the value of score function at theta w.r.t dispersion coefficient tau
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(dim) with the values of score function
                    w.r.t tau
        """
        gradient = np.zeros(self.dim)
        for i in range(self.dim):
            sd = np.log(np.exp(self.sigma[i]) + self.a)
            gradient[i] = - 1/(2 * self.tau[i]) + (((theta[i] - self.mu[i]) / (self.tau[i] * sd)) ** 2) / 2
            
        return gradient

class gamma_mean_field_family_lambda_param_overdisp:
    """Class for a mean field overdispersed gamma family of arbitrary, parametrized with mean and standard deviation
    
        lambda_alfa = log(exp(alfa) - a)
        lambda_beta = log(exp(alfa) - a)
    
        where alpha corresponds to the mean of the gamma distribution and beta corresponds to the std of the gamma dist
      
      Gamma parametrization:
          
          f(x|alpha, beta) = \frac{beta ^ alpha}{Gamma(alpha)} * x ^{alpha - 1} * \exp{- beta * x}
          
          E(X) = alpha / beta
          Var(X) = alpha / (beta^2)
          
    """
    
    def __init__(self, param, dim, tau):
        """The parameters for the gamma mean field family are passed as a dictionary
        
        Args:
            param: A dictionaray containing two entries with arrays of dimension (dim,)
                 - param["alpha"] is a 1-dim Ndarray with the np.log(alpha) parameters of independent Gamma distributions
                 - param["beta"] is a 1-dim Ndarray with the np.log(beta) parameters of independent Gamma distributions
                 - param["a"] see above
            tau: Overdispersion parameter value
        """
        
        self.alpha = param["alpha"]
        self.beta = param["beta"]
        self.a = param["a"]
        self.tau = tau
        self.dim = dim
        
    def sample(self, n_size):
        """Samples from the gamma mean field variational family
        
        Note: np.random.gamma has parametrization with shape = alpha, scale = 1 / beta
        
        Args:
            n_size: the number of samples to be generated
        Returns:
            samle: 2 dimensional NDarray where each row is a single sample from the variational family
        """
        
        
        sample = np.zeros((n_size, self.dim))
        for i in range(self.dim):
            mu = np.log(np.exp(self.alpha[i]) + self.a)
            sigma = np.log(np.exp(self.beta[i]) + self.a)
            alpha = (float(mu/sigma) ** 2 + self.tau[i] - 1) / self.tau[i]
            beta = float(mu/ (sigma**2)) / self.tau[i]
            tfd = tfp.distributions
            tfdGamma = tfd.Gamma(concentration=alpha, rate=beta)
            samples = tfdGamma.sample(n_size)
            sample[:, i] = samples.numpy()
        return sample
    
    def log_pdf(self, theta):
        """Calculates the value of log density of the mean field gamma family at theta
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log density will be evaluated
        Returns:
            log_dens: The value of log density at theta
        """
        
        log_dens = 0
        for i in range(self.dim):
            
            mu = np.log(np.exp(self.alpha[i]) + self.a)
            sigma = np.log(np.exp(self.beta[i]) + self.a)
            alpha = (float(mu/sigma) ** 2 + self.tau[i] - 1) / self.tau[i]
            beta = float(mu/ (sigma**2)) / self.tau[i]
            tfd = tfp.distributions
            tfdGamma = tfd.Gamma(concentration=alpha, rate=beta)
            log_dens = log_dens + float(tfdGamma.log_prob(theta[i]))
        return log_dens
    
    
    def grad_log_pdf_tau(self, theta):
        """"Calculate the value of score function at theta w.r.t dispersion coefficient tau
        
        Args:
            theta: 1-dim Ndarray of theta values at which the log gradient will be evaluated
        Returns:
            gradient: 1-dim Ndarray of len(dim) with the values of score function
                    w.r.t tau
        """
        
        gradient = np.zeros(self.dim)
        for i in range(self.dim):    
            mu = np.log(np.exp(self.alpha[i]) + self.a)
            sigma = np.log(np.exp(self.beta[i]) + self.a)
            alpha = (float(mu/sigma) ** 2)
            beta = float(mu/ (sigma**2))
            
            gradient[i] = (self.tau[i] * np.log(beta / self.tau[i]) - (alpha + self.tau[i] - 1) + beta * theta[i] - \
                          np.log(beta / self.tau[i]) * (alpha + self.tau[i] - 1) + sp.special.digamma((alpha + self.tau[i] - 1) /self.tau[i]) * (alpha - 1) - \
                          np.log(theta[i]) * (alpha - 1)) / self.tau[i]      
        return gradient
##### END of distribution class definition


##### GP model means class definintions    
class delta_mean:
    def __init__(self, hyperparametrs):
        """General class representing the mean function of model discrepancy GP
        
        Args:
            hyperparameters: A dictionary of hyperparameters for the mean
        """
        self.hyperparameters = hyperparametrs
        
    def zero(self, x):
        """Zero mean: m(x) = 0
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
        Returns: 0
        """
        return 0
    
    def constant(self, x):
        """Constant mean: m(x) = \Beta
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
            self.hyperparameters: A scalar with dictionary key "Beta"
        Returns: self.hyperparameters["Beta"]
        """
        return self.hyperparameters["Beta"]
    
    def linear(self, x):
        """Linear mean: m(x) =Intercept + \Beta * x^T
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
            self.hyperparameters: A scalar with dictionary key "Intercept" and a vector of parameters
                                  with dictionary key "Beta" of the same length as lengt(x)
        Returns:
            mean: Intercept + \Beta * (x, theta)^T
        """
        mean = self.hyperparameters["Intercept"] + np.dot(self.hyperparameters["Beta"], x)
        return mean
    
    
class model_mean:
    def __init__(self, hyperparametrs):
        """General class representing the mean function of computer model GP
        
        Args:
            hyperparameters: A dictionary of hyperparameters for the mean
        """
        self.hyperparameters = hyperparametrs
        
    def zero(self, x, theta):
        """Zero mean: m(x,theta) = 0
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
            theta: 1-dim Ndarray (or scalar) of calibration parameters  
        Returns: 0
        """
        return 0
    
    def constant(self, x, theta):
        """Constant mean: m(x,theta) = \Beta
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
            theta: 1-dim Ndarray (or scalar) of calibration parameters
            self.hyperparameters: A scalar with dictionary key "Beta"
        Returns: self.hyperparameters["Beta"]
        """
        return self.hyperparameters["Beta"]
    
    def dot_product(self, x, theta):
        """Dot product mean, for two dimensional model input x as defined in the 
           simulation setup in Kejzlar and Maiti (2020): 
           
           m(x,theta) = \Beta * (\theta_1 * cos(x_1) + \theta_2 * cos(x_2))
        
        Args:
            x: 1-dim Ndarray of model imputs (or scalar)
            theta: 1-dim Ndarray (or scalar) of calibration parameter
            self.hyperparameters: A scalar with dictionary key "Beta"
        Returns: self.hyperparameters["Beta"] * np.dot(x, theta)
        """
        if x.ndim == 1:
            return self.hyperparameters["Beta"] * np.dot(np.array([np.cos(x[0]), np.sin(x[1])]), theta)
        else:
            return self.hyperparameters["Beta"] * np.dot(np.array([np.cos(x[:,0]), np.sin(x[:,1])]), theta)
##### END of GP model means class definitnon
    
##### Adaptive learning rates definitnion AdaGrad, Adam, RMSProp    
def AdaGrad(diag_G, gradient, eta = 0.01, eps = 1 / (10 ** 8)):
    """Adaptive Learning rate computation using AdaGrad.
    
    Args:
        diag_G: Diagonal of the sum of squared gradients from previous steps
        gradient: The value of the gradiant in the current step of SGA
        eta: Innitial step size
        eps: Small constant to prevent division by 0
        
    Returns:
        rho: Step size for SGA
        diag_G_next: Diagonal of the sum of squared gradients from previous steps + gradient ** 2 
    """
    
    diag_G_next = diag_G + gradient ** 2
    rho = eta / np.sqrt(eps + diag_G_next)   
    return rho, diag_G_next

def RMSProp(avg_old, gradient, decay = 1 / 2, eta = 0.01 , eps = 1 / (10 ** 6)):
    """Adaptive learning rate computation using RMSProp.
    
    Args:
        avg_old: The value decay * avg_old + (1 - decay) * gradient ** 2 from previous step
        gradient: The value of the gradiant in the current step of SGA
        decay: The value of forgetting factor
        eta: Innitial step size
        eps: Small constant to prevent division by 0
        
    Returns:
        rho: Step size for SGA
        avg_next: The value of decay * avg_old + (1 - decay) * gradient ** 2 for the currents step
    """
    avg_new =  decay * avg_old + (1 - decay) * gradient ** 2
    rho =  eta / np.sqrt(avg_new + eps)
    return rho, avg_new

def Adam(m_old, v_old, gradient, step_counter, decay = 1 / 2, eta = 0.01 , eps = 1 / (10 ** 6)):
    """Calculates the proposed change in the value of parameters that are being optimized with SGA using
       the Adam algorithim, i.e. computes the value of "proposal", where param_new = param_old + proposal
    
    Args:
        m_old: The value decay * m_old + (1 - decay) * gradient from previous step
        v_old: The value decay * m_old + (1 - decay) * gradient ** 2 from previous step
        gradient: The value of the gradiant in the current step of SGA
        step_counter: The current step value in the SGA
        decay: The value of forgetting factor
        eta: Innitial step size
        eps: Small constant to prevent division by 0
        
    Returns:
        proposal: Teh proposed "addition" to the parameters value in SGA
        m_old: The value decay * m_old + (1 - decay) * gradient from the current step
        v_old: The value decay * m_old + (1 - decay) * gradient ** 2 from the current step
    """
    
    m_new = decay * m_old + (1 - decay) * gradient
    v_new = decay * v_old + (1 - decay) * gradient ** 2
    m_new_hat = m_new / (1 - decay ** step_counter)
    v_new_hat = v_new / (1 - decay ** step_counter)
    proposal = eta * m_new_hat / np.sqrt(v_new_hat + eps)
    return proposal, m_new, v_new
##### END of adaptive learning rates defitnitons

# Variational Calibration - Rao-Blackwellization

In [12]:
##### This chunk containts the main interface functions for VC with Rao-Blackwellization only
##### however, it also has some utilities for other variance reduction functions, so run this code
##### irrespective the version of VC

def variational_doctioinary_to_lambda(variational_dictionary):
    """Function takes a dictionary of variational mean field family and transforms it into a 1-dim Ndarray
    of that represents a gradient of log pdfs. The oredr in the gradient is as follows:
    [theat.mu, theta.sigma, sigma.alpha, sigma.beta, kernel_f_l.alpha, kernel_f_l.beta,
    kernel_f_eta.alpha, kernel_f_eta.beta, kernel_delta_l.alpha, kernel_delta_l.beta, kernel_delta_eta.alpha,
    kernel_delta_eta.beta] and then [mean_f.mu, mean_f.sigma, mean_delta.mu, mean_delta.sigma] if the mean 
    hyperparameters are not fixed.
    
    Args:
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
    Return:
        log_variational_grad: A 1-dim NDarray of of gradient of log variational pdf where the gradients are in
                            the order: theta, sigma, kernel_f_l, kernel_f_eta, kernel_delta_l, kernel_delta_eta,
                            mean_f, mean_delta
        indexes: A list of strings that outlines what value in log_variational_grad corresponds to which part 
                of the GP specifications.
        
    """
    indexes = []
    ### Theta + kernels
    var_lambda = variational_dictionary["theta"].mu
    if "kernel_f_l" in variational_dictionary:
        var_lambda = np.concatenate([var_lambda, variational_dictionary["theta"].sigma,
                                     variational_dictionary["sigma"].alpha,
                                     variational_dictionary["sigma"].beta,
                                     variational_dictionary["kernel_f_l"].alpha,
                                     variational_dictionary["kernel_f_l"].beta,
                                     variational_dictionary["kernel_f_eta"].alpha,
                                     variational_dictionary["kernel_f_eta"].beta,
                                     variational_dictionary["kernel_delta_l"].alpha,
                                     variational_dictionary["kernel_delta_l"].beta,
                                     variational_dictionary["kernel_delta_eta"].alpha,
                                     variational_dictionary["kernel_delta_eta"].beta])
        
        indexes = indexes + ["theta"] * variational_dictionary["theta"].dim * 2 + \
                        ["sigma"] * variational_dictionary["sigma"].dim * 2 + \
                        ["kernel_f_l"] * variational_dictionary["kernel_f_l"].dim * 2 + \
                        ["kernel_f_eta"] * variational_dictionary["kernel_f_eta"].dim * 2 + \
                        ["kernel_delta_l"] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                        ["kernel_delta_eta"] * variational_dictionary["kernel_delta_eta"].dim * 2
    else:
        var_lambda = np.concatenate([var_lambda, variational_dictionary["theta"].sigma,
                                     variational_dictionary["sigma"].alpha,
                                     variational_dictionary["sigma"].beta,
                                     variational_dictionary["kernel_delta_l"].alpha,
                                     variational_dictionary["kernel_delta_l"].beta,
                                     variational_dictionary["kernel_delta_eta"].alpha,
                                     variational_dictionary["kernel_delta_eta"].beta])
        indexes = indexes + ["theta"] * variational_dictionary["theta"].dim * 2 + \
                        ["sigma"] * variational_dictionary["sigma"].dim * 2 + \
                        ["kernel_delta_l"] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                        ["kernel_delta_eta"] * variational_dictionary["kernel_delta_eta"].dim * 2
        

    ### Means
    if "mean_f" in variational_dictionary:
        var_lambda = np.concatenate([var_lambda, variational_dictionary["mean_f"].mu,
                                         variational_dictionary["mean_f"].sigma])   
    if "mean_delta" in variational_dictionary:
        var_lambda = np.concatenate([var_lambda,  variational_dictionary["mean_delta"].mu,
                                         variational_dictionary["mean_delta"].sigma])
    ## Indexes  


    if "mean_f" in variational_dictionary:
        indexes = indexes + ["mean_f"] * variational_dictionary["mean_f"].dim * 2
    if "mean_delta" in variational_dictionary:
        indexes = indexes + ["mean_delta"] * variational_dictionary["mean_delta"].dim * 2
    
    return var_lambda.flatten(), indexes

def ELBO_mask_SGA(i_low_type, i_hig_type, ELBO, variational_dictionary):
    """Function calculates the mask of ELBO so that the noisy ELBO gradient estimates are unbiased,
       based on the type of argument passed onto the function, i.e. sets ELBO = 0 for appropriate 
       elements of the elbo, for example if i_low_type = i_hig_type = 0, the elements of 
       delta GP are not updated
    
    Args:
        i_low_type: 1 for y, 0 for z
        i_hig_type: 1 for y, 0 for z
        ELBO: An array corresponding to the ELBO
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.                      
    Return:
        ELBO_new: Adjusted ELBO
    """
    
    indexes = []
    
    ## Indexes  
        
    if (i_low_type == 1) & (i_hig_type == 0):
        indexes = indexes + [1] * variational_dictionary["theta"].dim * 2 + \
                        [1] * variational_dictionary["sigma"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_l"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_eta"].dim * 2 + \
                        [1] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                        [1] * variational_dictionary["kernel_delta_eta"].dim * 2

        if "mean_f" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_f"].dim * 2
        if "mean_delta" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_delta"].dim * 2

    elif (i_low_type == 0) & (i_hig_type == 1):   
        indexes = indexes + [1] * variational_dictionary["theta"].dim * 2 + \
                        [1] * variational_dictionary["sigma"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_l"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_eta"].dim * 2 + \
                        [1] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                        [1] * variational_dictionary["kernel_delta_eta"].dim * 2

        if "mean_f" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_f"].dim * 2
        if "mean_delta" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_delta"].dim * 2


    elif (i_low_type == 0) & (i_hig_type == 0):
        indexes = indexes + [0] * variational_dictionary["theta"].dim * 2 + \
                        [0] * variational_dictionary["sigma"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_l"].dim * 2 + \
                        [1] * variational_dictionary["kernel_f_eta"].dim * 2 + \
                        [0] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                        [0] * variational_dictionary["kernel_delta_eta"].dim * 2

        if "mean_f" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_f"].dim * 2
        if "mean_delta" in variational_dictionary:
            indexes = indexes + [0] * variational_dictionary["mean_delta"].dim * 2


    elif (i_low_type == 1) & (i_hig_type == 1):
        
        if "kernel_f_l" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["theta"].dim * 2 + \
                            [1] * variational_dictionary["sigma"].dim * 2 + \
                            [1] * variational_dictionary["kernel_f_l"].dim * 2 + \
                            [1] * variational_dictionary["kernel_f_eta"].dim * 2 + \
                            [1] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                            [1] * variational_dictionary["kernel_delta_eta"].dim * 2
        else:
            indexes = indexes + [1] * variational_dictionary["theta"].dim * 2 + \
                            [1] * variational_dictionary["sigma"].dim * 2 + \
                            [1] * variational_dictionary["kernel_delta_l"].dim * 2 + \
                            [1] * variational_dictionary["kernel_delta_eta"].dim * 2
            

        if "mean_f" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_f"].dim * 2
        if "mean_delta" in variational_dictionary:
            indexes = indexes + [1] * variational_dictionary["mean_delta"].dim * 2
            
    ELBO_new = ELBO * np.array(indexes)
    return ELBO_new


def var_lambda_to_variational_dictionary(variational_dictionary, var_lambda, indexes):
    """Function takes the current the the updated value of variational parameters from the
    upcoming step in the SGA and updates the variational dictionary with these values.
    
    Args:
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        var_lambda: A 1-dim NDarray of updated values of variational parameters in order [theat.mu, theta.sigma,
                    sigma.alpha, sigma.beta, kernel_f_l.alpha, kernel_f_l.beta,
                    kernel_f_eta.alpha, kernel_f_eta.beta, kernel_delta_l.alpha, kernel_delta_l.beta, kernel_delta_eta.alpha,
                    kernel_delta_eta.beta] and then [mean_f.mu, mean_f.sigma, mean_delta.mu, mean_delta.sigma] if the mean 
                    hyperparameters are not fixed.
        indexes: A list of strings that outlines what value in var_lambda corresponds to which part 
                of the GP specifications.
    Return:
        variational_dictionary: Updated variational dictionary
    """
    var_lambda_pd = pd.DataFrame(var_lambda.reshape(1, len(indexes)), columns=indexes)

    # Theta
    dim = variational_dictionary["theta"].dim
    variational_dictionary["theta"].mu = np.array(var_lambda_pd["theta"])[0,:dim].astype(float)
    variational_dictionary["theta"].sigma = np.array(var_lambda_pd["theta"])[0,dim:].astype(float)
    # Noise
    variational_dictionary["sigma"].alpha = np.array(var_lambda_pd["sigma"])[0,:1].astype(float)
    variational_dictionary["sigma"].beta = np.array(var_lambda_pd["sigma"])[0,1:].astype(float)
    # kernel_f
    if "kernel_f_l" in variational_dictionary:
        dim = variational_dictionary["kernel_f_l"].dim
        variational_dictionary["kernel_f_l"].alpha = np.array(var_lambda_pd["kernel_f_l"])[0, :dim].astype(float)
        variational_dictionary["kernel_f_l"].beta = np.array(var_lambda_pd["kernel_f_l"])[0, dim:].astype(float)

        variational_dictionary["kernel_f_eta"].alpha = np.array(var_lambda_pd["kernel_f_eta"])[0,:1].astype(float)
        variational_dictionary["kernel_f_eta"].beta = np.array(var_lambda_pd["kernel_f_eta"])[0,1:].astype(float)
    # kernel_delta
    dim = variational_dictionary["kernel_delta_l"].dim
    variational_dictionary["kernel_delta_l"].alpha = np.array(var_lambda_pd["kernel_delta_l"])[0, :dim].astype(float)
    variational_dictionary["kernel_delta_l"].beta = np.array(var_lambda_pd["kernel_delta_l"])[0, dim:].astype(float)

    variational_dictionary["kernel_delta_eta"].alpha = np.array(var_lambda_pd["kernel_delta_eta"])[0,:1].astype(float)
    variational_dictionary["kernel_delta_eta"].beta = np.array(var_lambda_pd["kernel_delta_eta"])[0,1:].astype(float)

    if "mean_f" in variational_dictionary:
        dim = variational_dictionary["mean_f"].dim
        variational_dictionary["mean_f"].mu = np.array(var_lambda_pd["mean_f"])[0,:dim].astype(float)
        variational_dictionary["mean_f"].sigma = np.array(var_lambda_pd["mean_f"])[0,dim:].astype(float)
    if "mean_delta" in variational_dictionary:
        dim = variational_dictionary["mean_delta"].dim
        variational_dictionary["mean_delta"].mu = np.array(var_lambda_pd["mean_delta"])[0,:dim].astype(float)
        variational_dictionary["mean_delta"].sigma = np.array(var_lambda_pd["mean_delta"])[0,dim:].astype(float)
    
    return variational_dictionary

def delta_ELBO_hyper_priors_variational_RB(priors_dictionary, variational_dictionary, theta_sample_dictionary):
    """Function calculate the values of log_priors_pdf, log_variational_pdf, and log_variational_grad, given
       the current value of variational parameters.
       
    Args:
        theta_sample_dictionary: A sample from the variational family as a dictionary containing the keys
                                {"theta", "sigma", "kernel_f_l", "kernel_f_eta", "kernel_delta_l", "kernel_delta_eta"}
                                and "mean_f" or "mean_delta" if the mean hyperparameters are not fixed.
        priors_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
    Return:
        log_priors_pdf: A float scalar of the logarithm of priors pdf for all the parameters.
        log_variational_pdf: A float scalar of the logarithm of variational mean field family pdf
                            for all the parameters.
        log_variational_grad: A 1-dim NDarray of of gradient of log variational pdf where the gradients are in
                            the order: theta, sigma, kernel_f_l, kernel_f_eta, kernel_delta_l, kernel_delta_eta,
                            mean_f, mean_delta
    """
    log_priors_pdf = np.repeat(priors_dictionary["theta"].log_pdf(theta_sample_dictionary["theta"]), priors_dictionary["theta"].dim * 2)  
    log_variational_pdf = np.repeat(variational_dictionary["theta"].log_pdf(theta_sample_dictionary["theta"]), variational_dictionary["theta"].dim * 2)
    # Going over all the priors
    ### Theta prior
    if "kernel_f_l" in priors_dictionary:
        log_priors_pdf = np.concatenate([log_priors_pdf, np.repeat(priors_dictionary["sigma"].log_pdf(theta_sample_dictionary["sigma"]), priors_dictionary["sigma"].dim * 2), 
                                         np.repeat(priors_dictionary["kernel_f_l"].log_pdf(theta_sample_dictionary["kernel_f_l"]),priors_dictionary["kernel_f_l"].dim * 2) ,
                                         np.repeat(priors_dictionary["kernel_f_eta"].log_pdf(theta_sample_dictionary["kernel_f_eta"]), priors_dictionary["kernel_f_eta"].dim * 2) ,
                                         np.repeat(priors_dictionary["kernel_delta_l"].log_pdf(theta_sample_dictionary["kernel_delta_l"]), priors_dictionary["kernel_delta_l"].dim * 2),
                                         np.repeat(priors_dictionary["kernel_delta_eta"].log_pdf(theta_sample_dictionary["kernel_delta_eta"]), priors_dictionary["kernel_delta_eta"].dim*2)])
    else:
        log_priors_pdf = np.concatenate([log_priors_pdf, np.repeat(priors_dictionary["sigma"].log_pdf(theta_sample_dictionary["sigma"]), priors_dictionary["sigma"].dim * 2), 
                                     np.repeat(priors_dictionary["kernel_delta_l"].log_pdf(theta_sample_dictionary["kernel_delta_l"]), priors_dictionary["kernel_delta_l"].dim * 2),
                                     np.repeat(priors_dictionary["kernel_delta_eta"].log_pdf(theta_sample_dictionary["kernel_delta_eta"]), priors_dictionary["kernel_delta_eta"].dim*2)])
    ### Means
    log_priors_pdf.flatten()
    if "mean_f" in priors_dictionary:
        log_priors_pdf = np.concatenate([log_priors_pdf, np.repeat(priors_dictionary["mean_f"].log_pdf(theta_sample_dictionary["mean_f"]), priors_dictionary["mean_f"].dim * 2)])   
    if "mean_delta" in priors_dictionary:
        log_priors_pdf = np.concatenate([log_priors_pdf, np.repeat(priors_dictionary["mean_delta"].log_pdf(theta_sample_dictionary["mean_delta"]), priors_dictionary["mean_delta"].dim * 2)])   
 
    if "kernel_f_l" in priors_dictionary:
        log_variational_pdf= np.concatenate([log_variational_pdf, np.repeat(variational_dictionary["sigma"].log_pdf(theta_sample_dictionary["sigma"]), variational_dictionary["sigma"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_f_l"].log_pdf(theta_sample_dictionary["kernel_f_l"]), variational_dictionary["kernel_f_l"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_f_eta"].log_pdf(theta_sample_dictionary["kernel_f_eta"]), variational_dictionary["kernel_f_eta"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_delta_l"].log_pdf(theta_sample_dictionary["kernel_delta_l"]), variational_dictionary["kernel_delta_l"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_delta_eta"].log_pdf(theta_sample_dictionary["kernel_delta_eta"]), variational_dictionary["kernel_delta_eta"].dim * 2)])
    else:
        log_variational_pdf= np.concatenate([log_variational_pdf, np.repeat(variational_dictionary["sigma"].log_pdf(theta_sample_dictionary["sigma"]), variational_dictionary["sigma"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_delta_l"].log_pdf(theta_sample_dictionary["kernel_delta_l"]), variational_dictionary["kernel_delta_l"].dim * 2),
                                                                np.repeat(variational_dictionary["kernel_delta_eta"].log_pdf(theta_sample_dictionary["kernel_delta_eta"]), variational_dictionary["kernel_delta_eta"].dim * 2)])
    
    ### Means
    log_variational_pdf.flatten()
    if "mean_f" in priors_dictionary:
        log_variational_pdf = np.concatenate([log_variational_pdf, np.repeat(variational_dictionary["mean_f"].log_pdf(theta_sample_dictionary["mean_f"]), variational_dictionary["mean_f"].dim*2)])   
    if "mean_delta" in priors_dictionary:
        log_variational_pdf = np.concatenate([log_variational_pdf, np.repeat(variational_dictionary["mean_delta"].log_pdf(theta_sample_dictionary["mean_delta"]), variational_dictionary["mean_delta"].dim*2)])   
 
    ########### Gradient handling log_variational_grad
    ### Theta
    log_variational_grad = variational_dictionary["theta"].grad_log_pdf(theta_sample_dictionary["theta"]) 

    ### Noise + other scales
    if "kernel_f_l" in variational_dictionary:
        log_variational_grad = np.concatenate([log_variational_grad, variational_dictionary["sigma"].grad_log_pdf(theta_sample_dictionary["sigma"]),
                                                                variational_dictionary["kernel_f_l"].grad_log_pdf(theta_sample_dictionary["kernel_f_l"]),
                                                                variational_dictionary["kernel_f_eta"].grad_log_pdf(theta_sample_dictionary["kernel_f_eta"]),
                                                                variational_dictionary["kernel_delta_l"].grad_log_pdf(theta_sample_dictionary["kernel_delta_l"]),
                                                                variational_dictionary["kernel_delta_eta"].grad_log_pdf(theta_sample_dictionary["kernel_delta_eta"])])
    else:
        log_variational_grad = np.concatenate([log_variational_grad, variational_dictionary["sigma"].grad_log_pdf(theta_sample_dictionary["sigma"]),
                                                                variational_dictionary["kernel_delta_l"].grad_log_pdf(theta_sample_dictionary["kernel_delta_l"]),
                                                                variational_dictionary["kernel_delta_eta"].grad_log_pdf(theta_sample_dictionary["kernel_delta_eta"])])
    
    ### Means
    log_variational_grad.flatten()
    if "mean_f" in priors_dictionary:
        log_variational_grad = np.concatenate([log_variational_grad, variational_dictionary["mean_f"].grad_log_pdf(theta_sample_dictionary["mean_f"])])   
    if "mean_delta" in priors_dictionary:
        log_variational_grad = np.concatenate([log_variational_grad, variational_dictionary["mean_delta"].grad_log_pdf(theta_sample_dictionary["mean_delta"])])   
 
    return log_priors_pdf, log_variational_pdf, log_variational_grad

def vine_p_RB(i, j, l, n, data_input, kernels_input, means_input, vine_type = "C"): #Works for general case
    """Function calculates the log copula density, and log likelihoods used to compute tilde p
    in algorithm 2 (as defined in Kejzlar and Maiti (2020))
    
    Note:
    - Rao-Blackwellizaed Version
    - this is the "p" for the Algorithm 1 and for Algorithm 1 + Control Variates 
    - p under Rao-Blackwellization defined below
    
    Args:
        i: Index i from the bijection
        j: Index j from the bijection
        l: Truncation level
        n: Sample size
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        kernels_input: A dictionary containing 3 items: f, delta, and noise kernels
        means_input: A dictionary containing 2 items: f, delta
        vine_type: "C" or "D" depending on which wine copule we want to consider
        
    Returns:
        log_c: log bivariate copula density
        log_pdf_low: log likelihood of datapoint with smaller index
        log_pdf_hig: log likelihood of datapoint with larger index
    
    """
    
    if vine_type == "D":
        if j > l:
            c_i_j = 1.0
        else:
            c_i_j = bivariate_gauss_copula(i, j, data_input, kernels_input, means_input, vine_type = "D")

        # definition of a_i multiplicatior
        a = 2 * l
        if i <= l:
            a = a - (l + 1 - i)
        if i > (n - l):
            a = a - (l - n + i)

        # definition of b_{i+j} multiplicator
        b = 2 * l
        if (i + j) <= l:
            b = b - (l + 1 - j - i)
        if (i + j) > (n - l):
            b = b - (l - n + j + i)

        # Standardized data
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type  = std_transform(i, j + i, data_input, kernels_input, means_input)
    else:
        if j > l:
            c_i_j = 1.0
        else:
            c_i_j = bivariate_gauss_copula(i, j, data_input, kernels_input, means_input, vine_type = "C")

        # definition of a_i multiplier 
        a = n - 1
        # definition of b_{j+i}
        b = l
        if (j + i) <= l:
            b = b + (n - 1 - l)
              
        # Standardized data
        Y_low_std, Y_hig_std, var_low, var_hig, i_low_type, i_hig_type = std_transform(j, j + i, data_input, kernels_input, means_input)
        
    a = float(a)
    b = float(b)
    

    log_c = np.log(c_i_j)
    log_pdf_low = (1 / a) * np.log(sp.stats.norm.pdf(Y_low_std))
    log_pdf_hig = (1 / b) * np.log(sp.stats.norm.pdf(Y_hig_std))

    return log_c, log_pdf_low, log_pdf_hig

def vine_p_to_p_eval(log_c, log_pdf_low, log_pdf_hig, i_low_type, i_hig_type, priors_dictionary, means_input = None):
    """Function takes output from vine_p_RB and turns it into the value tilde "p" in algorithm 2 (as defined in Kejzlar and Maiti (2020))
    
    Args:
        log_c: log bivariate copula density
        log_pdf_low: log likelihood of datapoint with smaller index
        log_pdf_hig: log likelihood of datapoint with larger index
        i_low_type: 1 for y, 0 for z
        i_hig_type: 1 for y, 0 for z
        priors_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        means_input: A dictionary containing 2 items: f, delta
        
    Returns:
        vine_p: Value of tilde "p"
    """
    
    if (i_low_type == 1) & (i_hig_type == 0):
        if means_input != None:
            if (means_input["f"].__name__ == "zero") or (means_input["f"].__name__ == "constant"):
                vine_p_theta = log_c
            else:
                vine_p_theta = log_c + log_pdf_low
        vine_p_sigma = log_c + log_pdf_low
        vine_p_f_l = log_c
        vine_p_f_eta = log_c + log_pdf_hig + log_pdf_low
        vine_p_delta_l = log_c
        vine_p_delta_eta = log_c + log_pdf_low
        
        if "mean_delta" in variational_dictionary:
            vine_p_beta = log_c + log_pdf_low
            

    elif (i_low_type == 0) & (i_hig_type == 1):
        if means_input != None:
            if (means_input["f"].__name__ == "zero") or (means_input["f"].__name__ == "constant"):
                 vine_p_theta = log_c
            else:
                vine_p_theta = log_c + log_pdf_hig
        vine_p_sigma = log_c + log_pdf_hig
        vine_p_f_l = log_c
        vine_p_f_eta = log_c + log_pdf_hig + log_pdf_low
        vine_p_delta_l = log_c
        vine_p_delta_eta = log_c + log_pdf_hig
        
        if "mean_delta" in variational_dictionary:
            vine_p_beta = log_c + log_pdf_hig



    elif (i_low_type == 0) & (i_hig_type == 0):
        vine_p_theta = 0
        vine_p_sigma = 0
        vine_p_f_l = log_c
        vine_p_f_eta = log_c + log_pdf_hig + log_pdf_low
        vine_p_delta_l = 0
        vine_p_delta_eta = 0
        
        if "mean_delta" in variational_dictionary:
            vine_p_beta = 0


    elif (i_low_type == 1) & (i_hig_type == 1):
        if means_input != None:
            if (means_input["f"].__name__ == "zero") or (means_input["f"].__name__ == "constant"):
                vine_p_theta = log_c
            else:
                vine_p_theta = log_c + log_pdf_low + log_pdf_hig
        vine_p_sigma = log_c + log_pdf_low + log_pdf_hig
        if "kernel_f_l" in priors_dictionary:
            vine_p_f_l = log_c
            vine_p_f_eta = log_c + log_pdf_hig + log_pdf_low
        vine_p_delta_l = log_c
        vine_p_delta_eta = log_c + log_pdf_low + log_pdf_hig
        
        if "mean_delta" in variational_dictionary:
            vine_p_beta = log_c + log_pdf_low + log_pdf_hig
            
    vine_p = np.repeat(vine_p_theta, priors_dictionary["theta"].dim * 2)
    
    if "mean_f" in variational_dictionary:
            vine_p_beta_f = log_c + log_pdf_low + log_pdf_hig
    
    if "kernel_f_l" in priors_dictionary:
        vine_p = np.concatenate([vine_p, np.repeat(vine_p_sigma, priors_dictionary["sigma"].dim * 2), 
                                         np.repeat(vine_p_f_l,priors_dictionary["kernel_f_l"].dim * 2) ,
                                         np.repeat(vine_p_f_eta, priors_dictionary["kernel_f_eta"].dim * 2) ,
                                         np.repeat(vine_p_delta_l, priors_dictionary["kernel_delta_l"].dim * 2),
                                         np.repeat(vine_p_delta_eta, priors_dictionary["kernel_delta_eta"].dim*2)])
    else:
        vine_p = np.concatenate([vine_p, np.repeat(vine_p_sigma, priors_dictionary["sigma"].dim * 2), 
                                         np.repeat(vine_p_delta_l, priors_dictionary["kernel_delta_l"].dim * 2),
                                         np.repeat(vine_p_delta_eta, priors_dictionary["kernel_delta_eta"].dim*2)])
    vine_p.flatten()
    
    if "mean_f" in priors_dictionary:
        vine_p = np.concatenate([vine_p, np.repeat(vine_p_beta_f, priors_dictionary["mean_f"].dim * 2)])
    
    if "mean_delta" in priors_dictionary:
        vine_p = np.concatenate([vine_p, np.repeat(vine_p_beta, priors_dictionary["mean_delta"].dim * 2)])
        
    return vine_p

def delta_ELBO_hyper_RB(data_input, Y_index, Z_index, input_m, means, kernels,
               vine_type, theta_sample_dictionary, priors_dictionary, variational_dictionary,
               n_obs, x_dim, theta_dim, i, j, l, n):
    """Function calculates the gradient of ELBO only with Rao-Blackwellization, i.e. withou control variates
    or importance sampling. It works for a general variational family because the specifics are abstracted 
    into level above.
    
    Args:
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        Y_index: Index of data points in range 1,..., len(Y)
        Z_index: Index of data points in range len(Y) + 1, n
        input_m: Variable inputs and parameters form model evaluations, each row is (x, theta)
        kernels: A dictionary containing 3 items: {"f": string describing the type of kernel,
                 "delta": string describing the type of kernel, "noise": current noise value as float}
        means: A dictionary containing 2 items: f, delta
               {"f": string desciribing the type of mean, "delta": string describing the type of mean}
               note: if there are no priors present for the mean parameters, the dictionary contains instances 
               of model_mean and delta_mean classes
        vine_type: "C" or "D" depending on which wine copule we want to consider
        theta_sample_dictionary: A sample from the variational family as a dictionary containing the keys
                                {"theta", "sigma", "kernel_f_l", "kernel_f_eta", "kernel_delta_l", "kernel_delta_eta"}
                                and "mean_f" or "mean_delta" if the mean hyperparameters are not fixed.
        priors_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        n_obs: The number of experimental observations.
        x_dim: The dimension of model inputs.
        theta_dim: The dimension of calibraiton parameters.
        i: Index i from the bijection
        j: Index j from the bijection
        l: Truncation level
        n: Total number of data, i.e. n_obs + n_m
        
    Returns:
        Elbo_add: The value of gradient of ELBO evaluated at current sample from variational family.
                  It is in the form of 1 dim Ndarray vector. In case the computation fails, it is a vector
                  of np.nan values.
        h: The value of log_variational_grad (nothing is done with it here in this function). In case 
           the computation of gradient of ELBO fails, it is a vector of np.nan values.
           - NOTE: this is an artifact here, but used in the CV version
    """
    # How does theta_smaple_dictionary work:
    # the key in the dictionary is what part of the GP specification it is related with a list under the key
    # if it is any other key than "theta", the first item in the list is what kind of type of kernel/mean it is
    # the rest is the current value of hyperparameters
    
    
    kernel_type = kernels["f"]
    if kernel_type == "sq_quad":
        kernel_f = float(theta_sample_dictionary["kernel_f_eta"]) ** 2 * skl.RBF(length_scale = theta_sample_dictionary["kernel_f_l"])       
    elif kernel_type == "matern":
        kernel_f = float(theta_sample_dictionary["kernel_f_eta"]) ** 2 * skl.Matern(length_scale = theta_sample_dictionary["kernel_f_l"], nu = 1.5)
    elif kernel_type == "zero":
        kernel_f = 0
    # kernel delta
    kernel_type = kernels["delta"]
    if kernel_type == "sq_quad":
        kernel_delta = float(theta_sample_dictionary["kernel_delta_eta"]) ** 2 * skl.RBF(length_scale = theta_sample_dictionary["kernel_delta_l"])       
    elif kernel_type == "matern":
        kernel_delta = float(theta_sample_dictionary["kernel_delta_eta"]) ** 2 * skl.Matern(length_scale = theta_sample_dictionary["kernel_delta_l"],
                                                               nu = 1.5)
    elif kernel_type == "zero":
        kernel_delta = 0
    
    noise = theta_sample_dictionary["sigma"][0]
    kernels_input = {"f":kernel_f, "delta": kernel_delta, "sigma": float(noise)}
    # mean_f
    if "mean_f" in priors_dictionary: # Is mean fixed or not?
        mean_type = means["f"]
        if mean_type != "linear":
            mean_hyperparameters = {"Beta": float(theta_sample_dictionary["mean_f"])}
            mean_f_init = model_mean(mean_hyperparameters)
            
            if mean_type == "constant":
                mean_f = mean_f_init.constant
            elif mean_type == "dot_product":
                mean_f = mean_f_init.dot_product
            elif mean_type == "LDM":
                mean_f = mean_f_init.LDM
            
        else:
            # Linear mean
            mean_hyperparameters = {"Intercep": float(theta_sample_dictionary["mean_f"][1]),
                                    "Beta": float(theta_sample_dictionary["mean_f"][2:])}  
            mean_f_init = model_mean(mean_hyperparameters)
            mean_f = mean_f_init.linear
    else:
        mean_f = means["f"]
        
    # mean_delta
    if "mean_delta" in priors_dictionary: # Is mean fixed or not?
        mean_type = means["delta"]
        if mean_type != "linear":
            mean_hyperparameters = {"Beta": float(theta_sample_dictionary["mean_delta"])}
            mean_delta_init = delta_mean(mean_hyperparameters)
            mean_delta = mean_delta_init.constant # No dot product version here            
        else:
            # Linear mean
            mean_hyperparameters = {"Intercep": float(theta_sample_dictionary["mean_delta"][1]),
                                    "Beta": float(theta_sample_dictionary["mean_delta"][2:])}  
            mean_delta_init = delta_mean(mean_hyperparameters)
            mean_delta = mean_delta_init.linear
    else:
        mean_delta = means["delta"]
        
    means_input = {"f": mean_f, "delta": mean_delta}

                   
    log_priors_pdf, log_variational_pdf, log_variational_grad = delta_ELBO_hyper_priors_variational_RB(priors_dictionary,
                                                                                                    variational_dictionary,
                                                                                                    theta_sample_dictionary)  
    
    theta_variational = np.repeat(theta_sample_dictionary["theta"], n_obs)
    # This works for dim of theta > 1
    theta_variational = theta_variational.reshape((n_obs, theta_dim), order='F')
    
    Y_theta = np.concatenate((Y_index, theta_variational), axis = 1)
    if np.any(input_m != None):
        Z_theta = np.concatenate((Z_index, input_m[:,x_dim:]), axis = 1)
        Theta_input = np.concatenate((Y_theta, Z_theta))
    else:
        Theta_input = Y_theta
    
    data_input["Theta"] = Theta_input
    
    log_c, log_pdf_low, log_pdf_hig = vine_p_RB(i, j, l, n, data_input, kernels_input, means_input, vine_type = vine_type)

    if vine_type == "D":
        i_low = i
        i_hig = i + j
    elif vine_type == "C":
        i_low = j
        i_hig = i + j
            
    i_low_type = data_input["Response"][i_low - 1,:][1]
    i_hig_type = data_input["Response"][i_hig - 1,:][1]
    
    p_eval = vine_p_to_p_eval(log_c, log_pdf_low, log_pdf_hig, i_low_type, i_hig_type, priors_dictionary, means_input = means_input)
    
    
    if (np.all(np.isnan(p_eval)) != True) & (np.all(np.isfinite(np.array([p_eval, log_priors_pdf, log_variational_pdf]))) == True):    
        ELBO_add = int(round(l * (2 * n - (l + 1)) / 2)) * log_variational_grad * p_eval
        ELBO_add = ELBO_add - log_variational_grad * (log_variational_pdf - log_priors_pdf)
        h = log_variational_grad 
        if np.isnan(ELBO_add).all() != True:
            return ELBO_add, h
        else:
            return np.repeat(np.nan, len(log_variational_grad)), np.repeat(np.nan, len(log_variational_grad))         
    else:
        return np.repeat(np.nan, len(log_variational_grad)), np.repeat(np.nan, len(log_variational_grad))
    
##### The following function is the main intervace for Variational Calibration

def VC_calibration_hyper_RB(data_input, Y_index, Z_index, input_m, kernels,
                         means, vine_type, priors_dictionary, variational_dictionary, x_dim, theta_dim, n_steps,
                         S, l, folder, learning_rate = "RMSProp", n_core = 1, eta = 0.01, decay = 1 / 2):
    """Scalable variational calibration of computer models according to Kejzlar and Maiti (2020).
    
    - Rao-Blackwellization ONLY
    
    Args:
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        Y_index: Index of data points in range 1,..., len(Y)
        Z_index: Index of data points in range len(Y) + 1, n
        input_m: Ndarray with model inputs.
        kernels: A dictionary containing 3 items: {"f": string describing the type of kernel,
                 "delta": string describing the type of kernel, "noise": current noise value as float}
        means: A dictionary containing 2 items: f, delta
               {"f": string desciribing the type of mean, "delta": string describing the type of mean}
               note: if there are no priors present for the mean parameters, the dictionary contains instances 
               of model_mean and delta_mean classes
        vine_type: "C" or "D" depending on which wine copule we want to consider
        theta_sample_dictionary: A sample from the variational family as a dictionary containing the keys
                                {"theta", "sigma", "kernel_f_l", "kernel_f_eta", "kernel_delta_l", "kernel_delta_eta"}
                                and "mean_f" or "mean_delta" if the mean hyperparameters are not fixed.
        priors_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        x_dim: The dimension of model inputs.
        theta_dim: The dimension of calibraiton parameters.
        n_steps: The number of evolutions of SGA to be performed.
        S: Numer of samples from the variational family to be taken in the MCMC approximation of ELBO
        l: Truncation level
        folder: The string with path to the folder where the results would be stored
                - stores array of variatioanal parameters and time stamps
                - saves every 10 iterations - this can be modified at the end of the function def.
        learning_rate: The learning reate algorithm to be used in SGA: "Basic," "AdaGrad," "RMSProp," or "Adam"
        n_core: The number of cores to be used for the parallel evaluation of MCMC integrals
        eta: Innitial learning rate
        decay: The value of forgetting factor to be used in either "RMSProp" or "Adam"
    Returns:
        time_counter: An array of system time of each evolution step of the SGA
        var_lambda_array: A Pandas DataFrame of the evolution of variational parameters during the optimization.
                          Each row correspond to a single evolution step
    """
    
    time_counter = np.array(time.monotonic())
    
    n_m = input_m.shape[0]
    n_obs = data_input["Response"].shape[0] - n_m
    n = n_m + n_obs
    bijection_domain = int(round(l * (2 * n - (l + 1)) / 2))
    
    ##### Innitial variational parameters
    var_lambda, indexes = variational_doctioinary_to_lambda(variational_dictionary)
    

    var_lambda_array = pd.DataFrame(var_lambda.reshape(1, len(indexes)), columns=indexes)
    # Creating folder to save results
    if not os.path.exists(folder):
        os.makedirs(folder)
    else:
        shutil.rmtree(folder)           #removes all the subdirectories!
        os.makedirs(folder)
    
    # Learning rate preset
    if learning_rate == "RMSProp":
        # RMSProp preset
        eps = 1 / (10 ** 6)
        avg_old = np.zeros(len(var_lambda))
    elif learning_rate == "AdaGrad":   
        # AdaGrad preset to default values
        eps = 1 / (10 ** 6)
        diag_G = np.zeros(len(var_lambda))
    elif learning_rate == "Basic":
    # Fixed step size - Do Not reccomend to use
        #eps = 0.05
        eps = eta
        alpha = - 0.75
        t = np.round((eps / bijection_domain) ** (1 / alpha)) # step counter
        rho = t ** (alpha)
    elif learning_rate == "Adam":
        eps = 1 / (10 ** 6)
        m_old = np.zeros(len(var_lambda))
        v_old = np.zeros(len(var_lambda))
        step_counter = 1
    
    for steps in tqdm(range(n_steps), desc = "SGA optimization"):
        
        # Bijection
        k = np.random.randint(1, bijection_domain + 1)
        i, j = bijection_array(k, n, l)

        
        #### Variational defintion family at the current value of the variational parameter
        variational_dictionary = var_lambda_to_variational_dictionary(variational_dictionary, var_lambda, indexes)
        
        #### Sampling from the variational family for MCMC approximation 
        theta_sample_dictionary_MCMC = []
        for s in range(S):
            theta_sample_dictionary = {}
            for item in variational_dictionary:
                theta_sample_dictionary[item] = variational_dictionary[item].sample(1).flatten()
            theta_sample_dictionary_MCMC = theta_sample_dictionary_MCMC + [theta_sample_dictionary]
        
        ELBO = 0
        ELBO_add_array, h_array = zip(*Parallel(n_jobs = n_core)(delayed(delta_ELBO_hyper_RB)(data_input, Y_index,
                                                                                           Z_index, input_m, means,
                                                                                           kernels, vine_type,
                                                                                           theta_sample_dictionary,
                                                                                           priors_dictionary,
                                                                                           variational_dictionary,
                                                                                           n_obs, x_dim, theta_dim, i,
                                                                                           j, l, n) for theta_sample_dictionary in theta_sample_dictionary_MCMC))
        ELBO_add_array = np.asarray(ELBO_add_array).T
        h_array = np.asarray(h_array).T
                
        nan_mask = ~np.all(np.isnan(h_array), axis = 0)
        h_array = h_array[:, nan_mask]
        ELBO_add_array = ELBO_add_array[:, nan_mask]
        mean_S = h_array.shape[1]

        # Estimate of control variete
        num = np.sum((h_array - np.mean(h_array, axis= 1)[:,None]) * (ELBO_add_array - np.mean(ELBO_add_array, axis= 1)[:,None]), axis = 1) / (ELBO_add_array.shape[1] - 1)
        denom = np.var(h_array, axis= 1)
        a = 0
        ELBO = (np.sum(ELBO_add_array, axis = 1) - a * np.sum(h_array, axis = 1)) / mean_S
        
        # Here we will judge the the types of data
        ############################################
        if vine_type == "D":
            i_low = i
            i_hig = i + j
        elif vine_type == "C":
            i_low = j
            i_hig = i + j
            
        i_low_type = data_input["Response"][i_low - 1,:][1]
        i_hig_type = data_input["Response"][i_hig - 1,:][1]
        
        
        ELBO = ELBO_mask_SGA(i_low_type, i_hig_type, ELBO, variational_dictionary)
        if (mean_S != 0) and (~np.all(np.isnan(ELBO))):
            if learning_rate == "RMSProp":
                rho, avg_new = RMSProp(avg_old = avg_old, gradient = ELBO, decay = decay, eta = eta , eps = eps)
            elif learning_rate == "AdaGrad":
                rho, diag_G_prop = AdaGrad(diag_G = diag_G, gradient = ELBO, eta = eta, eps = eps)
            
            if learning_rate == "Adam":
                proposal, m_new, v_new = Adam(m_old = m_old, v_old = v_old, gradient = ELBO, step_counter = step_counter,
                                              decay = decay, eta = eta , eps = eps)
                var_lambda_prop = var_lambda + proposal
            else:
                var_lambda_prop = var_lambda + rho * ELBO
            var_lambda = var_lambda_prop
            
            
            time_counter = np.append(time_counter, time.monotonic())
                
            if learning_rate == "RMSProp":
                    #RMSProp
                avg_old = avg_new
            elif learning_rate == "AdaGrad":
                    # AdaGrad
                diag_G = diag_G_prop
            elif learning_rate == "Basic":
                    # stepsize update
                t = t + 1
                rho = t ** (alpha)
            elif learning_rate == "Adam":
                m_old = m_new
                v_old = v_new
                step_counter = step_counter + 1
        else:
            print("here")
            time_counter = np.append(time_counter, time.monotonic())            
        
        var_lambda_array = var_lambda_array.append(pd.DataFrame(var_lambda.reshape(1, len(indexes)), columns=indexes))
        # Change this modulo setting based how often you want the samples to be saved
        if (steps % 10) == 9:
            np.save(os.getcwd() + "/" + folder + "/" + "param_nobs_" + str(n_obs) + "_S_" + str(S) + "_l_" + str(l) +  "_vine_" + vine_type + "_step_" + str(n_steps), var_lambda_array)
            np.save(os.getcwd() + "/" + folder + "/" + "time_nobs_" + str(n_obs) + "_S_" + str(S) + "_l_" + str(l) + "_vine_" + vine_type + "_step_" + str(n_steps), time_counter)
    
    return time_counter, var_lambda_array

# Variational Calibration - Rao-Blackwellization + Control Variates

In [14]:
#### Interface for Variational Calibration with Rao-Blackwellization and CV
# Make sure you run the chunk "Variational Calibration - Rao-Blackwellization" prior this because it contains
# some utility functions needed here

def VC_calibration_hyper_RB_CV(data_input, Y_index, Z_index, input_m, kernels,
                         means, vine_type, priors_dictionary, variational_dictionary, x_dim, theta_dim, n_steps,
                         S, S_CV, l, folder, learning_rate = "RMSProp", n_core = 1, eta = 0.01, decay = 1 / 2):
    """Scalable variational calibration of computer models according to Kejzlar and Maiti (2020).
    
    - Rao-Blackwellization + Control Variates
    
    Args:
        data_input: A dictionary containing 3 items: Response, X, theta for each of data points
        Y_index: Index of data points in range 1,..., len(Y)
        Z_index: Index of data points in range len(Y) + 1, n
        input_m: Ndarray with model inputs.
        kernels: A dictionary containing 3 items: {"f": string describing the type of kernel,
                 "delta": string describing the type of kernel, "noise": current noise value as float}
        means: A dictionary containing 2 items: f, delta
               {"f": string desciribing the type of mean, "delta": string describing the type of mean}
               note: if there are no priors present for the mean parameters, the dictionary contains instances 
               of model_mean and delta_mean classes
        vine_type: "C" or "D" depending on which wine copule we want to consider
        theta_sample_dictionary: A sample from the variational family as a dictionary containing the keys
                                {"theta", "sigma", "kernel_f_l", "kernel_f_eta", "kernel_delta_l", "kernel_delta_eta"}
                                and "mean_f" or "mean_delta" if the mean hyperparameters are not fixed.
        priors_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        variational_dictionary: A dictionary of instances of gamma_mean_field_family and gaussian_mean_field_family
                            in a dictionary with keys {"theta", "sigma", "kernel_f_l", "kernel_f_eta",
                            "kernel_delta_l", "kernel_delta_eta"} and "mean_f" or "mean_delta" if the
                            mean hyperparameters are not fixed.
        x_dim: The dimension of model inputs.
        theta_dim: The dimension of calibraiton parameters.
        n_steps: The number of evolutions of SGA to be performed.
        S: Numer of samples from the variational family to be taken in the MCMC approximation of ELBO
        S_cv: number of independet samples to be generated in order to compute control variates
        l: Truncation level
        folder: The string with path to the folder where the results would be stored
                - stores array of variatioanal parameters and time stamps
                - saves every 10 iterations - this can be modified at the end of the function def.
        learning_rate: The learning reate algorithm to be used in SGA: "Basic," "AdaGrad," "RMSProp," or "Adam"
        n_core: The number of cores to be used for the parallel evaluation of MCMC integrals
        eta: Innitial learning rate
        decay: The value of forgetting factor to be used in either "RMSProp" or "Adam"
    Returns:
        time_counter: An array of system time of each evolution step of the SGA
        var_lambda_array: A Pandas DataFrame of the evolution of variational parameters during the optimization.
                          Each row correspond to a single evolution step
    """
    
    time_counter = np.array(time.monotonic())
    
    n_m = input_m.shape[0]
    n_obs = data_input["Response"].shape[0] - n_m
    n = n_m + n_obs
    bijection_domain = int(round(l * (2 * n - (l + 1)) / 2))
    
    ##### Innitial variational parameters
    var_lambda, indexes = variational_doctioinary_to_lambda(variational_dictionary)
    var_lambda_array = pd.DataFrame(var_lambda.reshape(1, len(indexes)), columns=indexes)
    # Creating folder to save results
    if not os.path.exists(folder):
        os.makedirs(folder)
    else:
        shutil.rmtree(folder)           #removes all the subdirectories!
        os.makedirs(folder)
    
    # Learning rate preset
    if learning_rate == "RMSProp":
        # RMSProp preset
        eps = 1 / (10 ** 6)
        avg_old = np.zeros(len(var_lambda))
    elif learning_rate == "AdaGrad":   
        # AdaGrad preset to default values
        eps = 1 / (10 ** 6)
        diag_G = np.zeros(len(var_lambda))
    elif learning_rate == "Basic":
        # Fixed step size - Do not recommend
        eps = eta
        alpha = - 0.75
        t = np.round((eps / bijection_domain) ** (1 / alpha)) # step counter
        rho = t ** (alpha)
    elif learning_rate == "Adam":
        eps = 1 / (10 ** 6)
        m_old = np.zeros(len(var_lambda))
        v_old = np.zeros(len(var_lambda))
        step_counter = 1
    
    # the main VD algorithm
    eta_original = eta
    for steps in tqdm(range(n_steps), desc = "SGA optimization"):
        if steps < 10:
            eta = eta_original / 10
        else:
            eta = eta_original
        
        # Bijection
        k = np.random.randint(1, bijection_domain + 1)
        i, j = bijection_array(k, n, l)

        #### Variational defintion family at the current value of the variational parameter
        variational_dictionary = var_lambda_to_variational_dictionary(variational_dictionary, var_lambda, indexes)
        
        #### Sampling from the variational family for MCMC approximation 
        theta_sample_dictionary_MCMC = []
        for s in range(S + S_CV):
            theta_sample_dictionary = {}
            for item in variational_dictionary:
                theta_sample_dictionary[item] = variational_dictionary[item].sample(1).flatten()
            theta_sample_dictionary_MCMC = theta_sample_dictionary_MCMC + [theta_sample_dictionary]
        
        ### Kernels and means
        ELBO = 0
        ELBO_add_array, h_array = zip(*Parallel(n_jobs = n_core)(delayed(delta_ELBO_hyper_RB)(data_input, Y_index,
                                                                                           Z_index, input_m, means,
                                                                                           kernels, vine_type,
                                                                                           theta_sample_dictionary,
                                                                                           priors_dictionary,
                                                                                           variational_dictionary,
                                                                                           n_obs, x_dim, theta_dim, i,
                                                                                           j, l, n) for theta_sample_dictionary in theta_sample_dictionary_MCMC))
        
        # Control Variates handling
        ELBO_add_array = np.asarray(ELBO_add_array).T
        h_array = np.asarray(h_array).T
                
        nan_mask = ~np.all(np.isnan(h_array), axis = 0)
        h_array = h_array[:, nan_mask]
        ELBO_add_array = ELBO_add_array[:, nan_mask]
        
        # Separation of data into CV estimates part and ELBO estimates part
        h_array_CV = h_array[:,(-S_CV):]
        ELBO_add_array_CV = ELBO_add_array[:,(-S_CV):]
        h_array = h_array[:, :(-S_CV)]
        ELBO_add_array = ELBO_add_array[:, :(-S_CV)]
        
        mean_S = h_array.shape[1]

        # Estimate of control variete
        num = np.sum((h_array_CV - np.mean(h_array_CV, axis= 1)[:,None]) * (ELBO_add_array_CV - np.mean(ELBO_add_array_CV, axis= 1)[:,None]), axis = 1) / (ELBO_add_array_CV.shape[1] - 1)
        denom = np.var(h_array_CV, axis= 1)
        a = num / denom
        
        ELBO = (np.sum(ELBO_add_array, axis = 1) - a * np.sum(h_array, axis = 1)) / mean_S
        
        ############################################
        if vine_type == "D":
            i_low = i
            i_hig = i + j
        elif vine_type == "C":
            i_low = j
            i_hig = i + j
            
        i_low_type = data_input["Response"][i_low - 1,:][1]
        i_hig_type = data_input["Response"][i_hig - 1,:][1]
        
        
        ELBO = ELBO_mask_SGA(i_low_type, i_hig_type, ELBO, variational_dictionary)

        if (mean_S != 0) and (~np.all(np.isnan(ELBO))):
            if learning_rate == "RMSProp":
                rho, avg_new = RMSProp(avg_old = avg_old, gradient = ELBO, decay = decay, eta = eta , eps = eps)
            elif learning_rate == "AdaGrad":
                rho, diag_G_prop = AdaGrad(diag_G = diag_G, gradient = ELBO, eta = eta, eps = eps)
            
        # Validity check for proposal
            if learning_rate == "Adam":
                proposal, m_new, v_new = Adam(m_old = m_old, v_old = v_old, gradient = ELBO, step_counter = step_counter,
                                              decay = decay, eta = eta , eps = eps)
                var_lambda_prop = var_lambda + proposal
            else:
                var_lambda_prop = var_lambda + rho * ELBO
            var_lambda = var_lambda_prop
            
            
            time_counter = np.append(time_counter, time.monotonic())
                
            if learning_rate == "RMSProp":
                    #RMSProp
                avg_old = avg_new
            elif learning_rate == "AdaGrad":
                    # AdaGrad
                diag_G = diag_G_prop
            elif learning_rate == "Basic":
                    # stepsize update
                t = t + 1
                rho = t ** (alpha)
            elif learning_rate == "Adam":
                m_old = m_new
                v_old = v_new
                step_counter = step_counter + 1
        else:
            print("here")
            time_counter = np.append(time_counter, time.monotonic())
                
        # Change this modulo setting based how often you want the samples to be saved
        var_lambda_array = var_lambda_array.append(pd.DataFrame(var_lambda.reshape(1, len(indexes)), columns=indexes))
        if (steps % 10) == 9:
            np.save(os.getcwd() + "/" + folder + "/" + "param_nobs_" + str(n_obs) + "_S_" + str(S) + "_l_" + str(l) +  "_vine_" + vine_type + "_step_" + str(n_steps), var_lambda_array)
            np.save(os.getcwd() + "/" + folder + "/" + "time_nobs_" + str(n_obs) + "_S_" + str(S) + "_l_" + str(l) + "_vine_" + vine_type + "_step_" + str(n_steps), time_counter)
    
    return time_counter, var_lambda_array

# Variational Calibration - Rao-Blackwellization + Control Variates + Imoprtance Sampling